In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import seaborn as sb
import squidpy

In [ ]:
vis_data = "/home/workspace/EXP-00971/TIS05393-001-010-EXP-00971-SB005/outs"

adata = squidpy.read.visium(vis_data)

adata.var_names_make_unique()

sc.pp.calculate_qc_metrics(adata, inplace=True)
adata.var['mt'] = [gene.startswith('MT-') for gene in adata.var_names]
adata.obs['mt_frac'] = adata[:, adata.var['mt']].X.sum(1).A.squeeze()/adata.obs['total_counts']

In [ ]:
fig, axs = plt.subplots(1,4, figsize=(15,4))
fig.suptitle('Covariates for filtering')
sb.distplot(adata.obs['total_counts'], kde=False, ax = axs[0])
sb.distplot(adata.obs['total_counts'][adata.obs['total_counts']<10000], kde=False, bins=40, ax = axs[1])
sb.distplot(adata.obs['n_genes_by_counts'], kde=False, bins=60, ax = axs[2])
sb.distplot(adata.obs['n_genes_by_counts'][adata.obs['n_genes_by_counts']<4000], kde=False, bins=60, ax = axs[3])

In [ ]:
sc.pl.spatial(
    adata,
    img_key = "hires",
    color = ["total_counts", "n_genes_by_counts", "mt_frac"],
    vmax = "p99",
    wspace = 0
)

In [ ]:
sc.pl.spatial(
    adata,
    img_key = "hires",
    color = ["pct_counts_in_top_50_genes", "pct_counts_in_top_100_genes", "pct_counts_in_top_200_genes", "pct_counts_in_top_500_genes"],
    vmax = "p99",
    wspace = 0
)

In [ ]:
adata = adata[adata.obs['pct_counts_in_top_50_genes'] <= 95].copy()

sc.pl.spatial(
    adata,
    img_key = "hires",
    color = ["pct_counts_in_top_50_genes", "pct_counts_in_top_100_genes", "pct_counts_in_top_200_genes", "pct_counts_in_top_500_genes"],
    vmax = "p99",
    wspace = 0
)

In [ ]:
adata.write_h5ad('filtered_adata.h5ad')